In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method


"""

In [ ]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

In [ ]:
#so that it will have the adapter defined
from datajoint_utils import *

In [ ]:
test_mode = True

# Debugging the contains method

In [ ]:
import system_utils as su

In [ ]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

In [ ]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

import classification_utils as clu
import proofreading_utils as pru

import datajoint as dj

from pykdtree.kdtree import KDTree
import trimesh_utils as tu
import proofreading_utils as pru
import numpy as np

# Defining the Table

In [ ]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [ ]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

In [ ]:
# minnie,schema = du.configure_minnie_vm()
# minnie.AutoProofreadNeurons5.drop()
# minnie.AutoProofreadStats5.drop()
# minnie.AutoProofreadSynapse5.drop()
# minnie.AutoProofreadSynapseErrors5.drop()
# minnie.schema.external['faces'].delete(delete_external_files=True)
# minnie.schema.external['skeleton'].delete(delete_external_files=True)

# Creating the Auto Proofread Neuron Table

In [ ]:
manual_seg_ids = dj.U("segment_id") & (m65mat.Nucleus.Info * (m65mat.ProofreadSegment() & "ver=88"))
manual_seg_ids

In [ ]:
import numpy as np
import time
import classification_utils as clu


verbose = True

@schema
class ManualProofreadNeuronSkeletons(dj.Computed):
    definition="""
    -> minnie.Decomposition()
    split_index          : tinyint unsigned             # the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    ---
    neuron_skeleton: longblob    # the skeleton of the entire neuron
    """
    key_source = (minnie.Decomposition() & manual_seg_ids
                 & minnie.DecompositionAxon().proj() 
             ) 

    def make(self,key):
        """
        Pseudocode:
        1) Pull Down All of the Neurons
        2) Get the nucleus centers and the original mesh

        """
        
        # 1) Pull Down All of the Neurons
        segment_id = key["segment_id"]
        print(f"\n---- Working on {segment_id}----")
        whole_pass_time = time.time()
        
        neuron_objs,split_indexes = du.decomposition_with_spine_recalculation(segment_id)
        
        ManualProofreadNeuronSkeletons.insert([dict(key,
                                                    split_index = sp,
                                                    neuron_skeleton=n.skeleton) 
                                               for n,sp in zip(neuron_objs,split_indexes)],skip_duplicates=True)
            

        print(f"\n\n ***------ Total time for {key['segment_id']} = {time.time() - whole_pass_time} ------ ***")


# Running the Populate

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__manual_proofread_neuron_skeletons'")
(curr_table)#.delete()

In [ ]:
import time
pru = reload(pru)
nru = reload(nru)
import neuron_searching as ns
ns = reload(ns)
clu = reload(clu)
du = reload(du)
import random

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    ManualProofreadNeuronSkeletons.populate(reserve_jobs=True, suppress_errors=True, order="random")
else:
    ManualProofreadNeuronSkeletons.populate(reserve_jobs=True, suppress_errors=False, order="random")
print('Populate Done')

print(f"Total time for ManualProofreadNeuronSkeletons populate = {time.time() - start_time}")